导入NSL KDD数据作为评估决策树模型准确率的标准

获取数据

In [12]:
import pandas as pd
data = pd.read_csv("KDDTrain+.csv")
#data

筛选特征值和目标值

In [3]:
order = ["src_bytes","dst_host_srv_rerror_rate","dst_bytes","dst_host_same_src_port_rate","dst_host_srv_count","dst_host_rerror_rate","logged_in"]
x = data[order]
y = data["class"]

数据预处理无量纲化（标准化）

In [4]:
from sklearn.preprocessing import StandardScaler

transfer = StandardScaler()
data_new = transfer.fit_transform(x)

2.数据处理——缺失值处理
转换成字典

In [5]:
x = x.to_dict(orient="records")

字典特征抽取

In [6]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.tree import DecisionTreeClassifier,export_graphviz

In [7]:
transfer = DictVectorizer()
x = transfer.fit_transform(x)

导入模型

In [9]:
from sklearn.externals import joblib
estimator = joblib.load('DecisionTree.pkl')

结果预测

In [10]:
y_predict = estimator.predict(x)
print("预测流量结果为：\n", y_predict)
print("直接比对真实值和预测值:\n",y == y_predict)
score = estimator.score(x,y)
print("准确率为:\n",score)

预测流量结果为：
 ['normal.' 'normal.' 'neptune.' ... 'neptune.' 'neptune.' 'neptune.']
直接比对真实值和预测值:
 0        True
1        True
2        True
3        True
4        True
         ... 
25187    True
25188    True
25189    True
25190    True
25191    True
Name: class, Length: 25192, dtype: bool
准确率为:
 0.9915052397586536


对于待测数据，生成溯源所需数据并导出为.json文件

In [14]:
import json
def result_to_json(y_predict):
    out = []
    for i in range(0, len(y_predict)):
        if y_predict[i] != "normal.":
            handle = {}
            handle["type"] = "networkflow"
            handle["time"] = str(data[i:i+1]["time"].values[0])
            handle["ipsrc"] = data[i:i+1]["src_ip"].values[0]
            handle["ipdst"] = data[i:i+1]["dst_ip"].values[0]
            handle["attack_type"] = y_predict[i]
            handle["info"] = ""
            out.append(json.dumps(handle))
    return out

In [15]:
res = result_to_json(y_predict)
with open("abnormal-flows.json","w+") as f:
    for each in res:
        f.write(each)
        f.write("\n")
    f.close()